In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import math

In [8]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [9]:
# Hyperparameters
T = 300  # Total diffusion steps
beta_start = 1e-4
beta_end = 0.02
batch_size = 128
learning_rate = 2e-4
epochs = 10

In [10]:
class SimpleUnet(nn.Module):
    def __init__(self, channels = 1):
        super(SimpleUnet, self).__init__()
        self.encoder1 = nn.Conv2d(channels, 64, kernel_size=3, padding=1)
        self.encoder2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bottleneck = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.decoder2 = nn.ConvTranspose2d(128, 64, kernel_size=3, padding=1)
        self.decoder1 = nn.ConvTranspose2d(64, channels, kernel_size=3, padding=1)

    def forward(self, x, t):
        # For simplicity, we're not using timestep embedding
        x1 = F.relu(self.encoder1(x))
        x2 = F.relu(self.encoder2(x1))
        x_b = F.relu(self.bottleneck(x2))
        x_d2 = F.relu(self.decoder2(x_b))
        x_d1 = self.decoder1(x_d2)
        return x_d1